# dbApps09b Task: Transactions & Data Protection

**Course:** Database Applications Development (145085)  
**Institution:** Medina County Career Center

---

## Overview

In this lesson, you will:
- Understand database transactions and ACID properties
- Learn how to use BEGIN, COMMIT, and ROLLBACK
- Understand data protection laws (HIPAA, FERPA)
- Apply the principle of least privilege
- Write code that protects sensitive data

## Setup: Import Libraries & Create Database

In [ ]:
import sqlite3
import pandas as pd
from datetime import datetime

# Create connection to task09b database
conn = sqlite3.connect(":memory:")

# Enable foreign keys and autocommit OFF (so we control transactions)
conn.execute("PRAGMA foreign_keys = ON")
conn.isolation_level = None  # Manual transaction control

print("Database connection established.")
print("Manual transaction control enabled.")

## Create Tables: Bank Accounts & Audit Log

In [ ]:
# Create the bank_accounts table
createAccountsTable = """
CREATE TABLE bankAccounts (
    accountId INTEGER PRIMARY KEY,
    accountName TEXT NOT NULL,
    balance REAL NOT NULL
)
"""

conn.execute(createAccountsTable)

print("Bank accounts table created.")

In [ ]:
# Create the audit_log table
createAuditTable = """
CREATE TABLE auditLog (
    logId INTEGER PRIMARY KEY,
    accountId INTEGER NOT NULL,
    action TEXT NOT NULL,
    amount REAL,
    timestamp TEXT DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (accountId) REFERENCES bankAccounts(accountId)
)
"""

conn.execute(createAuditTable)

print("Audit log table created.")

In [ ]:
# Insert sample bank accounts
sampleAccounts = [
    (1, "Checking Account - John Doe", 5000.00),
    (2, "Savings Account - John Doe", 15000.00),
    (3, "Checking Account - Jane Smith", 8000.00),
    (4, "Savings Account - Jane Smith", 22000.00)
]

# Insert each account
for accountId, accountName, balance in sampleAccounts:
    conn.execute(
        "INSERT INTO bankAccounts (accountId, accountName, balance) VALUES (?, ?, ?)",
        (accountId, accountName, balance)
    )

conn.commit()

print(f"Inserted {len(sampleAccounts)} sample accounts.")

---

## Task 1: Display Current Account Balances

In [ ]:
# Display all bank accounts and their balances
dfAccounts = pd.read_sql("SELECT * FROM bankAccounts ORDER BY accountId", conn)
print("Bank Accounts (Initial State):")
print(dfAccounts)
print()
print(f"Total funds in system: ${dfAccounts['balance'].sum():.2f}")

---

## Task 2: Transaction - Simple Transfer (BEGIN / COMMIT)

### Instructions:
Write a transaction that:
1. Begins a transaction (BEGIN)
2. Transfers $200 from Account 1 to Account 2
3. Updates both account balances
4. Commits the transaction (COMMIT)
5. Verifies the new balances

In [ ]:
# Simple transfer transaction
transferAmount = 200
fromAccountId = 1
toAccountId = 2

print(f"Transferring ${transferAmount} from Account {fromAccountId} to Account {toAccountId}...")
print()

# BEGIN transaction
conn.execute("BEGIN")

# Withdraw from source account
conn.execute(
    "UPDATE bankAccounts SET balance = balance - ? WHERE accountId = ?",
    (transferAmount, fromAccountId)
)

# Deposit to destination account
conn.execute(
    "UPDATE bankAccounts SET balance = balance + ? WHERE accountId = ?",
    (transferAmount, toAccountId)
)

# COMMIT the transaction
conn.commit()

print("Transaction committed successfully.")
print()

In [ ]:
# Verify the transfer
dfAfterTransfer = pd.read_sql("SELECT * FROM bankAccounts ORDER BY accountId", conn)
print("Bank Accounts (After Transfer):")
print(dfAfterTransfer)
print()
print(f"Total funds in system: ${dfAfterTransfer['balance'].sum():.2f}")
print("Notice: The total is still the same. Money was moved, not created or destroyed.")

---

## Task 3: Transaction - Transfer with Audit Logging

### Instructions:
Write a transaction that:
1. Begins a transaction
2. Transfers $500 from Account 3 to Account 4
3. Logs BOTH operations in the audit_log table
4. Commits the transaction
5. Verifies both the accounts and audit log

In [ ]:
# Transfer with audit logging
transferAmount = 500
fromAccountId = 3
toAccountId = 4

print(f"Transferring ${transferAmount} from Account {fromAccountId} to Account {toAccountId}...")
print()

# BEGIN transaction
conn.execute("BEGIN")

# Withdraw from source account
conn.execute(
    "UPDATE bankAccounts SET balance = balance - ? WHERE accountId = ?",
    (transferAmount, fromAccountId)
)

# Log the withdrawal
conn.execute(
    "INSERT INTO auditLog (accountId, action, amount) VALUES (?, ?, ?)",
    (fromAccountId, "Withdrawal (Transfer Out)", transferAmount)
)

# Deposit to destination account
conn.execute(
    "UPDATE bankAccounts SET balance = balance + ? WHERE accountId = ?",
    (transferAmount, toAccountId)
)

# Log the deposit
conn.execute(
    "INSERT INTO auditLog (accountId, action, amount) VALUES (?, ?, ?)",
    (toAccountId, "Deposit (Transfer In)", transferAmount)
)

# COMMIT the entire transaction
conn.commit()

print("Transaction with audit logging committed successfully.")
print()

In [ ]:
# Verify the transfer
dfAccountsAfter = pd.read_sql("SELECT * FROM bankAccounts ORDER BY accountId", conn)
print("Bank Accounts (After Audit Transfer):")
print(dfAccountsAfter)
print()

# Verify the audit log
dfAuditLog = pd.read_sql("SELECT * FROM auditLog ORDER BY logId", conn)
print("Audit Log:")
print(dfAuditLog)

---

## Task 4: Transaction - ROLLBACK (Undo Changes)

### Instructions:
1. Show the current balance of Account 1
2. Start a transaction
3. Withdraw $1000 from Account 1
4. ROLLBACK (undo) the transaction
5. Verify the balance is unchanged

In [ ]:
# Show the balance BEFORE the rollback example
dfBefore = pd.read_sql("SELECT * FROM bankAccounts WHERE accountId = 1", conn)
balanceBefore = dfBefore['balance'].values[0]
print("Account 1 balance BEFORE transaction:")
print(f"${balanceBefore:.2f}")
print()

In [ ]:
# Start a transaction
print("Starting transaction...")
conn.execute("BEGIN")

# Withdraw $1000
withdrawalAmount = 1000
conn.execute(
    "UPDATE bankAccounts SET balance = balance - ? WHERE accountId = 1",
    (withdrawalAmount,)
)

# Check the balance within the transaction (before rollback)
dfDuringTransaction = pd.read_sql("SELECT * FROM bankAccounts WHERE accountId = 1", conn)
balanceDuring = dfDuringTransaction['balance'].values[0]
print(f"Account 1 balance DURING transaction (after withdrawal): ${balanceDuring:.2f}")
print()

# ROLLBACK the transaction
print("Rolling back the transaction...")
conn.execute("ROLLBACK")
print()

In [ ]:
# Verify the balance is unchanged
dfAfterRollback = pd.read_sql("SELECT * FROM bankAccounts WHERE accountId = 1", conn)
balanceAfter = dfAfterRollback['balance'].values[0]
print("Account 1 balance AFTER rollback:")
print(f"${balanceAfter:.2f}")
print()
print(f"Before: ${balanceBefore:.2f}")
print(f"After:  ${balanceAfter:.2f}")
print()
print("The balance is unchanged! ROLLBACK successfully undid the transaction.")

---

## Task 5: ACID Properties Explained

### Explain each ACID property in your own words, using the bank transfer as an example:

**A - Atomicity:**
"All or nothing" - A transaction is complete in its entirety or not at all.  
*Bank example:* When we transferred $200 from Account 1 to Account 2, BOTH the withdrawal and deposit happened, or NEITHER happened. We could never have a state where Account 1 was debited but Account 2 wasn't credited. If there was an error after the withdrawal, the ROLLBACK would undo it.

**C - Consistency:**
The database moves from one valid state to another valid state.  
*Bank example:* The total amount of money in the system must always stay the same (conservation of money). A $200 transfer should never create or destroy money. Before: $50,000 total. After: $50,000 total. The database remains in a consistent, valid state.

**I - Isolation:**
Transactions are independent and don't interfere with each other.  
*Bank example:* If two people are transferring money at the same time, their transactions are isolated. Person A's transfer doesn't interfere with Person B's transfer. Each transaction acts as if it's the only one running.

**D - Durability:**
Once committed, the changes are permanent and survive system failures.  
*Bank example:* Once we COMMIT the $200 transfer, the change is permanently written to disk. If the power goes out immediately after COMMIT, the transfer is still there when the system comes back up. If the power had gone out BEFORE COMMIT, the transaction wouldn't have happened at all.

---

## Task 6: HIPAA - Healthcare Data Protection

### Scenario:
A hospital database stores patient medical records, including diagnoses, medications, allergies, and treatments.

### Question:
What law protects this data, and what are a developer's responsibilities?

### Answer:

**The Law: HIPAA (Health Insurance Portability and Accountability Act)**
- Federal law in the United States that protects patient privacy
- Applies to healthcare providers, health plans, and healthcare clearinghouses
- Violations can result in fines up to $100 per violation (capped at $1.5 million per year)

**Developer's Responsibilities:**
1. **Encryption**: Encrypt patient data both at rest (in the database) and in transit (over the network)
2. **Access Control**: Implement authentication and authorization. Only authorized staff can see specific patient records
3. **Audit Trails**: Log all access to patient data. Who accessed what, when, and why?
4. **Data Minimization**: Only collect and store the minimum data needed
5. **Secure Deletion**: When data is no longer needed, securely delete it (not just a soft delete)
6. **Breach Notification**: Have a plan to notify patients if their data is breached
7. **User Discipline**: Never hardcode credentials, never log passwords, never print patient data to console
8. **Regular Updates**: Keep all software, libraries, and OS patches current

**Key Principle**: "Minimum Necessary" - Doctors should only see patient data relevant to treating that patient, not the entire medical history of the hospital.

---

## Task 7: FERPA - Education Data Protection

### Scenario:
A school database stores student records, including grades, test scores, disciplinary records, and special education plans.

### Question:
What law protects this data? Give 2 examples of how a database developer should protect it.

### Answer:

**The Law: FERPA (Family Educational Rights and Privacy Act)**
- Federal law that protects the privacy of student education records
- Gives parents/guardians the right to access their child's records
- Prevents unauthorized disclosure of student information
- Applies to all schools that receive federal funding (nearly all schools)
- Violations can result in loss of federal funding

**Example 1: Implement Role-Based Access Control**
- A teacher should only see grades for their own students
- A principal should see grades for all students in the school
- A student should see only their own grades (not other students' grades)
- A parent should see only their child's grades
- The database should enforce this at the query level, not just the UI
```
SELECT * FROM grades WHERE studentId = ? AND (userId = studentId OR userId = parentId OR userId = teacherId)
```

**Example 2: Audit All Data Access**
- Log every time someone accesses a student record (who, when, what they accessed)
- Create an audit trail in a secure audit_log table
- Include the username, timestamp, action (view, edit, delete), and student ID
- Regularly review the audit log for suspicious access patterns
- If someone accesses records without a legitimate educational reason, investigate immediately
```
INSERT INTO auditLog (userId, action, studentId, timestamp) VALUES (?, 'view_grades', ?, CURRENT_TIMESTAMP)
```

---

## Task 8: Principle of Least Privilege

### Question:
What is the "principle of least privilege"? How does it apply to database access?

### Answer:

**The Principle:**
A user should have the MINIMUM permissions necessary to perform their job. No more, no less.

**Why It Matters:**
- If someone's account is compromised (hacked), the attacker can only do what that user can do
- It limits the blast radius of mistakes (accidental data deletion)
- It enforces accountability (if data was deleted, we know who had permission to delete it)

**How It Applies to Database Access:**

1. **User Accounts in the Database**
   - Create a separate database user for each application role
   - App user account: Can SELECT and INSERT, but NOT DROP or DELETE
   - Admin account: Can SELECT, INSERT, UPDATE, DELETE, ALTER
   - Read-only account: Can SELECT only

2. **Table-Level Permissions**
   - Payroll staff: Can view the payroll table, but not the audit log
   - HR staff: Can view the employees table and edit some fields, but NOT salary data
   - Accountants: Can view all financial tables, but NOT employee passwords

3. **Row-Level Security**
   - A teacher can only see grades for their own classes
   - A regional manager can only see sales for their region
   - A nurse can only see patients they're assigned to, not all hospital patients

4. **Column-Level Permissions**
   - Employees can see colleague names and email, but NOT salary
   - Managers can see employee names and performance reviews, but NOT private notes

**Example in SQLite:**
```sql
-- Create a read-only database user
CREATE USER readonly_user IDENTIFIED BY 'secure_password';
GRANT SELECT ON products TO readonly_user;
-- This user can read products, but cannot insert, update, delete, or alter

-- Create an application user with limited permissions
CREATE USER app_user IDENTIFIED BY 'app_password';
GRANT SELECT, INSERT, UPDATE ON products TO app_user;
GRANT SELECT ON audit_log TO app_user;
-- This user can read and modify products, but cannot delete or view sensitive audit data
```

---

## Task 9: Transaction - Insert with Audit Logging

### Instructions:
Write a transaction that:
1. Inserts a new bank account
2. Logs the account creation in the audit_log table
3. Commits both operations atomically

In [ ]:
# Insert a new account with audit logging
newAccountId = 5
newAccountName = "Business Account - Smith Corp"
newBalance = 50000.00

print(f"Creating new account: {newAccountName}")
print()

# BEGIN transaction
conn.execute("BEGIN")

# Insert the new account
conn.execute(
    "INSERT INTO bankAccounts (accountId, accountName, balance) VALUES (?, ?, ?)",
    (newAccountId, newAccountName, newBalance)
)

# Log the account creation
conn.execute(
    "INSERT INTO auditLog (accountId, action, amount) VALUES (?, ?, ?)",
    (newAccountId, "Account Created", newBalance)
)

# COMMIT the transaction
conn.commit()

print("Transaction committed successfully.")
print()

In [ ]:
# Verify the new account and audit log entry
dfNewAccount = pd.read_sql("SELECT * FROM bankAccounts WHERE accountId = 5", conn)
print("New Account:")
print(dfNewAccount)
print()

dfAuditNew = pd.read_sql("SELECT * FROM auditLog WHERE accountId = 5", conn)
print("Audit Log Entry for New Account:")
print(dfAuditNew)

---

## Task 10: Reflection - Why Transactions Matter

### Question:
Why are transactions important for data integrity? Give a real-world scenario where missing a transaction could cause problems.

### Answer:

**Why Transactions Matter:**
Transactions ensure that multiple database operations either all succeed together or all fail together. They prevent partial updates that leave data in an inconsistent state.

**Real-World Scenario: The Missing Transaction**

Imagine an online store without transactions:

1. Customer buys a laptop for $1,000
2. Step 1: Add $1,000 to the seller's account balance
3. Step 2: Deduct $1,000 from the customer's account balance
4. Step 3: Insert a record in the "orders" table

If the system crashes after Step 1 but before Step 2 completes:
- The seller's account has +$1,000 (seller gets paid)
- The customer's account was NOT debited (customer keeps their money)
- The order was NOT recorded (store has no record of the sale)

**Result:** 
- Store lost $1,000
- Customer got the laptop for free
- No one can track what happened
- Data integrity is completely broken

**With a Transaction:**
All three steps are wrapped in a transaction. If any step fails, the entire transaction rolls back. Either:
- All three succeed and commit together, OR
- All three fail and rollback together
- No partial updates. No inconsistent state.

**Other Real-World Scenarios:**
- **Booking an airline seat**: Add passenger to flight, deduct seat from inventory, record payment. All or nothing.
- **Hospital medication**: Update patient dosage, record in patient chart, update pharmacy inventory. All or nothing.
- **Bank transfer**: Debit source account, credit destination account, create audit log. All or nothing.
- **Inventory management**: Record purchase order, deduct from warehouse stock, update supplier records. All or nothing.

**Key Insight:**
When you have multiple related database changes, ALWAYS wrap them in a transaction. Don't assume the system won't crash. Murphy's Law applies to databases: "Anything that can go wrong will go wrong at the worst possible time."

---

## Summary

You have learned:
- How to use BEGIN, COMMIT, and ROLLBACK to manage transactions
- The ACID properties that guarantee data integrity
- How HIPAA and FERPA protect sensitive data
- The principle of least privilege and how to implement it
- Why transactions are critical for real-world database applications

**Remember:** Transactions are not optional. They are fundamental to reliable database applications.